# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 1")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2108020.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1634546508720)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@15f34893


### Carga de CSV

In [2]:
val trade_details = spark.read.format("csv")
                              .option("header", "true")
                              .option("delimiter", ";")
                              .load("C:/Users/laura.serrano/Desktop/Spark for Data Engineers/reto1/trade_details.csv")

val trade_details_snapshot = spark.read.format("csv")
                                       .option("header", "true")
                                       .option("delimiter", ";")
                                       .load("C:/Users/laura.serrano/Desktop/Spark for Data Engineers/reto1/trade_details_snapshot.csv")

trade_details: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]
trade_details_snapshot: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


In [3]:
import spark.sqlContext.implicits._
val join_maturity= ((trade_details
                .where(($"mfamily" === "CURR") && ($"mgroup" === "FXD") && ($"mtype" === "SWLEG")))
          .join(trade_details_snapshot.withColumnRenamed("maturity", "maturity_null")
                                      .withColumnRenamed("origin_trade_number", "origin")
                                      .select("maturity_null", "origin")
                ,($"origin_trade_number" === $"origin") && $"maturity" === "NULL","left"))
          .select("mfamily", "mgroup", "mtype", "origin_trade_number","origin_contract_number","maturity", "maturity_null")

import spark.sqlContext.implicits._
join_maturity: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 5 more fields]


In [4]:
join_maturity.show()

+-------+------+-----+-------------------+----------------------+----------+-------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|maturity_null|
+-------+------+-----+-------------------+----------------------+----------+-------------+
|   CURR|   FXD|SWLEG|           19665185|              18622136|2020-04-29|         null|
|   CURR|   FXD|SWLEG|           19665186|              18622136|2020-12-30|         null|
|   CURR|   FXD|SWLEG|           19772399|              18724280|      NULL|   2020-11-05|
|   CURR|   FXD|SWLEG|           19772400|              18724280|2021-11-05|         null|
|   CURR|   FXD|SWLEG|           20980932|              19883451|      NULL|   2020-02-02|
|   CURR|   FXD|SWLEG|           20980933|              19883451|      NULL|   2020-01-19|
|   CURR|   FXD|SWLEG|           22798004|              21622649|      NULL|   2021-05-04|
|   CURR|   FXD|SWLEG|           22798005|              21622649|      NULL|   2021-05-11|

In [20]:
val final1= join_maturity.withColumn("maturity_null", when($"maturity_null".isNull,$"maturity")
                             .otherwise($"maturity_null"))

final1: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 5 more fields]


In [6]:
import org.apache.spark.sql.expressions.Window
val particion =  Window.partitionBy($"origin_contract_number").orderBy($"maturity_null".desc)

import org.apache.spark.sql.expressions.Window
particion: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@52aa0efa


In [22]:
val final_sin_duplicados = final1.withColumn("rn", row_number.over(particion)).where($"rn" === 1).drop("rn")
                           .drop($"maturity_null")

final_sin_duplicados: org.apache.spark.sql.DataFrame = [mfamily: string, mgroup: string ... 4 more fields]


In [23]:
final_sin_duplicados.show()

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|   CURR|   FXD|SWLEG|           20665178|              19622128|2020-12-30|
|   CURR|   FXD|SWLEG|           22798004|              21622649|      NULL|
|   CURR|   FXD|SWLEG|           20980932|              19883451|      NULL|
|   CURR|   FXD|SWLEG|           19665186|              18622136|2020-12-30|
|   CURR|   FXD|SWLEG|           19772399|              18724280|      NULL|
+-------+------+-----+-------------------+----------------------+----------+



In [29]:
val result_no_duplicates=trade_details.filter(!(($"mfamily" === "CURR") && ($"mgroup" === "FXD") && ($"mtype" === "SWLEG")))

result_no_duplicates: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mfamily: string, mgroup: string ... 4 more fields]


In [30]:
result_no_duplicates.show(22)

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|    IRD|    CF| null|           20513130|              19433281|2021-07-06|
|    IRD|    CF| null|           20533916|              19453781|2023-07-06|

### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.


In [31]:
val resultado =result_no_duplicates.union(final_sin_duplicados)

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [mfamily: string, mgroup: string ... 4 more fields]


Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

In [33]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()(24)(4) == "564367838")
assert(resultado.orderBy("origin_contract_number").collect()(19)(5) == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()(16)(5) == "NULL")